*Name: Pratik Warade *

# Organic Path Finding Using Artificial Intelligence

## Introduction 

Lawn can not only beautify the living environment, but also play an important role in solar radiation, climatic regulation, air purification, soil and water conservation and so on. Many of the big projects in cities or nearby has lawns which are sometimes screwed by humans as well as animals by walking continuously and indirectly taking the shortest path to reach their goals. Humans and animals are continuously engaged in finding path every day. But it's not because of a human or an animal. Many people have tendancy to follow the same path if their goals are same.
  
Following photo shows how students from Colorado State University are engaged in choosing a path from the lawn which goes towards the library, even though a cement pavement is available.
Following photo shows how students from Colorado State University are engaged in choosing a path from the lawn which goes towards the library, even though a cement pavement is available.
<img src= "http://www.cs.colostate.edu/~pswarade/pratik_imp/csu.jpg"style="max-width:100%; width: 50%">


In 1965, University of California at lrvine campus was build. According to larry wall article, the university has first bulid all the buliding but not the side walks. Instead the planted grass all around the campus and major building to get the traces of humans on grass. After a year, the build cement pavement based on this human behaviour.   

Following image shows the UCI famous park called as aldrich park. From this image, you can clearly see that the above story seem to be true as the park path does not have any pattern or kind geometry that usually seen in most of the latest construction. For example, most of the cement pavement are straight and angled at the edges. Looking at aldrich park, all the pathways are randomly generated and does follow human behaviour which we can say lead to goal with minimum cost.








<img src= "http://www.cs.colostate.edu/~pswarade/pratik_imp/aldrich_park.png"style="max-width:100%; width: 90%">

So how do we find such path? will it be feasible to always plant grass, then wait for a year and start construction of pavement?
What happens if urban/rural construction projects are completed with pavement build and no one is using and starts destroying lawns?  Well, artificial intelligence has a way to do same thing that UCI did i.e make people walk for a long time and get the path. 

In this Project, I am going use human behavior to address desire  path in  rural/urban development and transportation planning , also known as a bike trail, game trail, social trail, herd path, cow path, goat track, pig trail, use trail or bootleg trail.  AI agent will start walking from starting position to goal i.e ai agent will learns about human behavior and generate most efficient path.



## Data / code

What is Eighth Symposium on Educational Advances in Artificial Intelligence ( EAAI ) ?  

EAAI-18 provides a venue for researchers and educators to discuss pedagogical issues and share resources related to teaching and using AI in education across a variety of curricular levels (K-12 through postgraduate training), with an emphasis on undergraduate and graduate teaching and learning. The symposium seeks contributions showing how to more effectively learn about  AI, as well as how themes from AI may be used to enhance education more broadly .

For this project, I am going to use code from " Eighth Symposium on Educational Advances in Artificial Intelligence (EAAI) " which basically has a generic search algorithm  i.e breadth for search  and code that takes input as .bmp image  with a scale of 50 by 50 pixels.






Link to Code: http://modelai.gettysburg.edu/2017/pathfinding/organic-paths.zip 


## Methods 

Methods that will be used are : 

1) Best First Search  
2) A Star search  
3) Beam search

As said above, that given code has generic search algorithm i.e Breadth First Search with some implementation of above methods. Search algorithms metioned above can be programmed by figuring out the right variations of action cost g(s) and heuristic function h(s) and based on handling of expanded and unexpanded nodes. I am going to use hueristic functions such as manhattan distance, Euclidean distance and cost functions that will give required path. The detail explanation is given in below cells.

## Implimentation

In [14]:
# Required Imports

from sys import stdout
from operator import itemgetter
from subprocess import Popen, PIPE
from collections import deque
import random
import re
from heapq import heappop, heappush
from PIL import Image
import random
import math

Generic Search Algorithm ( Breadth For Search )

In [39]:
def search(start_state, depth_limit, heuristic, action_cost, openset_limit, is_goal_state,
           possible_transitions, make_graphviz_node):
    
    # states already checked
    closedset = []

    # a map of (state, action, time) => parent
    parents = {}
    #print("Start State, Parents",start_state,parents)
    # map of actual+estimated costs of going from every known state to goal state;
    # initialize with start state (F-score is simply the heuristic, no path cost)
    f_scores = {start_state: heuristic(start_state, 0, parents)}

    # map of actual costs of going from start state to every known state
    # initialize with start state (path cost = 0)
    g_scores = {start_state: 0}

    # states that we know exist but have not checked;
    # need both openset and openheap because heaps won't let us check if a state is a member
    openset = [start_state]
    openheap = [(f_scores[start_state], (start_state, None, 0))]

    # keep track of the largest openset we ever had to maintain
    max_openset_size = 1

    # while we still have a state to visit, and haven't found the goal
    while len(openset) > 0:

        # keep track of max_openset_size (just for benchmarking purposes)
        if max_openset_size < len(openset):
            max_openset_size = len(openset)

        # pick best state, make it the current state
        (val, (current_state, current_action, time)) = heappop(openheap)
        # if next-best state is just as good, we'll need to randomly choose
        equally_good = [(val, (current_state, current_action, time))]
        while len(openheap) > 0 and openheap[0][0] == val:
            (val2, (current_state2, current_action2, time2)) = heappop(openheap)
            equally_good.append((val2, (current_state2, current_action2, time2)))
        random.shuffle(equally_good)
        (val, (current_state, current_action, time)) = equally_good[0]
        # push all equally-good states not used
        for i in range(1, len(equally_good)):
            heappush(openheap, equally_good[i])

        openset.remove(current_state)

        # if we found the goal, we're done
        if is_goal_state(current_state):
            path = reconstruct_path(parents, (current_state, current_action, time))
            return (path, len(closedset), max_openset_size)

        # append current state to closed set so we don't visit it again
        closedset.append(current_state)

        # for each possible next state from current state
        transitions = possible_transitions(current_state)
        for (action, next_state) in transitions:

            # determine path cost to this next state
            tentative_g_score = g_scores[current_state] + action_cost(current_state, action, next_state)

            # skip next states that we've already visited and this path isn't cheaper
            if next_state in closedset and \
               (next_state not in g_scores or g_scores[next_state] <= tentative_g_score):
                continue

            # skip if we have a depth limit and this node is too deep
            if depth_limit is not None and time > depth_limit:
                closedset.append(next_state)
                continue

            # if this next state is new, or we found a cheaper way to get there, put it in the openset
            if next_state not in openset or tentative_g_score < g_scores[next_state]:

                # save the parentage and calculated scores for next state
                parents[(next_state, action, time+1)] = (current_state, current_action, time)
                g_scores[next_state] = tentative_g_score
                f_scores[next_state] = g_scores[next_state] + heuristic(next_state, time+1, parents)
                heappush(openheap, (f_scores[next_state], (next_state, action, time+1)))
                openset.append(next_state)

        # keep only best states if we have a limit (beam search / hillclimbing)
        if openset_limit is not None:
            new_openset = []
            new_openheap = []
            while len(new_openheap) < openset_limit and len(openheap) > 0:
                (val, (state, action, time)) = heappop(openheap)
                heappush(new_openheap, (val, (state, action, time)))
                new_openset.append(state)
            openset = new_openset
            openheap = new_openheap
            if len(openheap) == 1:
                #closedset = closedset[-openset_limit:] # keep only last N history
                closedset = closedset[-1:] # only one possible state to visit, may run out; so clear the history, allow covering old ground

        time += 1

    # if we made it here, we ran out of states to visit and never found the goal
    return (None, len(closedset), max_openset_size)


Calculate possible transition from any given point at terrain.Based on the direction of trasintion we will increment x value or y-vlaue as needded.Bellow function return possible value after transition with action assosciated with it as a list called transistion.

In [29]:
def possible_transitions((x, y)):
    global terrain_cost, width, height
    transitions = []
    if x > 0 and terrain_cost[(x-1,y)] >= 0:
        transitions.append(('west', (x-1, y)))
    if x < (width-1) and terrain_cost[(x+1,y)] >= 0:
        transitions.append(('east', (x+1, y)))
    if y > 0 and terrain_cost[(x,y-1)] >= 0:
        transitions.append(('north', (x, y-1)))
    if y < (height-1) and terrain_cost[(x,y+1)] >= 0:
        transitions.append(('south', (x, y+1)))
    if x < (width-1) and y < (height-1) and terrain_cost[(x+1,y+1)] >= 0:
        transitions.append(('southeast', (x+1, y+1)))
    if x > 0 and y < (height-1) and terrain_cost[(x-1,y+1)] >= 0:
        transitions.append(('soutwest', (x-1, y+1)))
    if x > 0 and y > 0 and terrain_cost[(x-1,y-1)] >= 0:
        transitions.append(('northwest', (x-1, y-1)))
    if x < (width-1) and y > 0 and terrain_cost[(x+1,y-1)] >= 0:
        transitions.append(('northeast', (x+1, y-1)))
    return transitions


Once we get path, we need to sort and reverse it and create a map that has its parent state and path.

In [30]:
def reconstruct_path(parents, (current_state, action, time)):
    path = [(current_state, action, time)]
    while path[-1] in parents:
        path.append(parents[path[-1]])
    path.reverse()
    return map(lambda (state, action, time): state, path)

Global Variables declaration. The map is represented as a two-dimensional array of integers; each integer in this array represents the cost of moving on that position in the map (high cost means slower movement). Possible actions are north, south, east, west, northeast, southeast, northwest, southwest. Some actions may not be allowed from a particular position if there is a barrier nearby (red in the map, value -1 in the 2D array of cost values). Refer to the file for details. Each map is actually 50x50 pixels,but generated .png files are enlarged one. Hence, we will declare Width and Height as 50. To store intial position of people we  will use init_x ana init_y as there (x,y) cordinates and goal_x and goal_y will be final position.

In [31]:
width = 50
height = 50

init_x = None
init_y = None
goal_x = None
goal_y = None

possible_initial_states = [] 
possible_goal_states = []

terrain_img = None
terrain_cost = {} # terrain cost is dictionary to get possible cost of terrain region based on (r,g,b) scale
terrain_cost_factor = 100 #just a multiplier
terrain_impassible = [] # cache of impassible locations

First job is to establish terrain and cost related to each of terrain region.To do this will go through each of the pixel i.e (x,y) cordinate that represent the terrain. Based on (r,g,b) values of pixel, we will assign weight to each terrain. i.e for grayscale cost is 191.This value represent the lawn where any one can walk or pass by. If it is red then that represents tree or any obstacle which a human or animal can't cross. Hence the -1 will be assigned to it. For green i.e a goal point cost is 0 and intial point is represented as blue which will be -1.

In [32]:
def establish_terrain(img_filename):
    global terrain_img, terrain_cost, terrain_impassible, width, height, possible_initial_states, possible_goal_states
    possible_initial_states = []
    possible_goal_states = []
    terrain_cost = {}
    terrain_img = Image.open(img_filename)
    terrain_impassible = []
    #get width and height of image which is 50*50
    (width, height) = terrain_img.size
    #call pixel class
    pixels = terrain_img.load()

    #go through each block in 50*50 image 
    for x in range(0, width):
        for y in range(0, height):
            #assign three values form pixel class to r,g,b
            (r,g,b) = pixels[x,y]
            
            #now assign cost to each block For e.g { ( 31,5 ) :191 )
            if r == g and g == b: # grayscale, terrain cost
                terrain_cost[(x,y)] = r
            elif r == 255 and g == 0 and b == 0: # red, impassible
                terrain_cost[(x,y)] = -1
                terrain_impassible.append((x,y))
            elif r == 0 and g == 255 and b == 0: # green, goal point
                possible_goal_states.append((x,y))
                terrain_cost[(x,y)] = 0
            elif r == 0 and g == 0 and b == 255: # blue, starting point
                possible_initial_states.append((x,y))
                terrain_cost[(x,y)] = -1
    #print(terrain_cost)
    
    
# Get path cost
def path_cost(path):
    cost = 0.0
    for i in range(len(path)-1):
        cost += action_cost(path[i][0], path[i][1], path[i+1][0])
    return cost

#update terrain cost
def update_terrain_costs(path):
    global terrain_cost
    for (x, y) in path:
        if terrain_cost[(x,y)] > 0:
            terrain_cost[(x,y)] = max(1, min(256, terrain_cost[(x,y)] - 1))
            

def make_graphviz_node((x, y), direction, visited = False):
    pass
           
            
#Draw terrain
def draw_terrain(out_filename):
    global terrain_img, terrain_cost
    pixels = terrain_img.load()
    for x in range(0, width):
        for y in range(0, height):
            c = int(terrain_cost[(x, y)])
            if c > 0:
                pixels[x, y] = (c, c, c)
    img = terrain_img.resize((width * 5, height * 5))
    img.save(out_filename)

In [40]:
#Make any random intial state from possible_intial_state list
def make_initial_state():
    global possible_initial_states, init_x, init_y
    (init_x, init_y) = random.choice(possible_initial_states)


In [41]:
# Calculate distance between two point on terrain
def dist((x1, y1), (x2, y2)):
    return ((x2-x1)**2 + (y2-y1)**2)**0.5


#Make any random goal state from possible_goal_state list
def make_goal_state():
    global possible_goal_states, goal_x, goal_y, init_x, init_y
    (goal_x, goal_y) = random.choice(possible_goal_states)
    while dist((init_x, init_y), (goal_x, goal_y)) < 10:
        (goal_x, goal_y) = random.choice(possible_goal_states)
        
#check for possible goal state match
def is_goal_state((x, y)):   
    return x == goal_x and y == goal_y

#### Beam Search

Even though the Breadth-First Search Algorithm is guaranteed to find the shortest path from a start node to a goal node in an unweighted graph, it is infeasible to use this algorithm on large search spaces because its memory consumption is exponential. This causes the algorithm run out of main memory before a solution can be found to most large, nontrivial problems. For this reason, Beam Search was developed in an attempt to achieve the optimal solution found by the Breadth-First Search Algorithm without consuming too much memory.

In order to accomplish this goal, Beam Search utilizes a heuristic function, h, to estimate the cost to reach the goal from a given node. It also uses a beam width, B, which specifies the number of nodes that are stored at each level of the Breadth-First Search. Thus, while the Breadth-First Search stores all the frontier nodes (the nodes connected to the closing vertices) in memory, the Beam Search Algorithm only stores the B nodes with the best heuristic values at each level of the search. The idea is that the heuristic function will allow the algorithm to select nodes that will lead it to the goal node, and the beam width will cause the algorithm to store only these important nodes in memory and avoid running out of memory before finding the goal state.

Instead of the open list used by the Breadth-First Search Algorithm, the Beam Search Algorithm uses the BEAM to store the nodes that are to be expanded in the next loop of the algorithm. A hash table is used to store nodes that have been visited, similar to the closed list used in the Breadth-First Search. Beam Search initially adds the starting node to the BEAM and the hash table. Then, each time through the main loop of the algorithm, Beam Search adds all of the nodes connected to the nodes in the BEAM to its SET of successor nodes and then adds the B nodes with the best heuristic values from the SET to the BEAM and the hash table. Note that a node that is already in the hash table is not added to the BEAM because a shorter path to that node has already been found. This process continues until the goal node is found, the hash table becomes full (indicating that the memory available has been exhausted), or the BEAM is empty after the main loop has completed (indicating a dead end in the search).

The Beam Search Algorithm is shown by the pseudocode below. This pseudocode assumes that the Beam Search is used on an unweighted graph so the variable g is used to keep track of the depth of the search, which is the cost of reaching a node at that level. ...by Andrew Jungwirth

### Hueristics

Heuristics is basically intuition. In short " A quick way to estimate how close we are to the goal". How close is a state to the goal..The heuristic function is based on the problem in concern. If the goal of a certain problem is finding the lowest distance path from one place to another, then the heuristic function is obviously the lowest distance from each node to the goal node. The problems in practice are not so simple and obvious most of the time. Therefore, it is essential to know some important aspects that should be considered when coming up with a good heuristic function to do the searching on that problem space. Some important pointers when designing a heuristic function are as follows.
It should never overestimate the true solution cost to the goal – For example, if the most feasible path to the goal node from the starting node costs ‘x’ no: of steps, then the heuristic function should not exceed that limit from any node for it to be a suitable heuristic function.
  
  
I have used Manhattan distance(bfs and a star search ) and Euclidean distance (Beam Search ).

##### Manhattan Distance: 

The standard heuristic for a square grid is the Manhattan distance. Look at cost function and find the
minimum cost D for moving from one space to an adjacent space. In the simple case, we can set D to
be 1.The heuristic on a square grid where you can move in 4 directions should be D times the Manhattan
distance:
  
Function heuristic(node)=   
dx=abs(node.x-goal.x)  
dy=abs(node.y-goal.y)  
return d*(dx+dy)  

Set D to the lowest cost between adjacent squares. In the absence of obstacles, and on terrain that has
the minimum movement cost D, moving one step closer to the goal should increase g by D and decrease
h by D. When we find f (which is set to g + h) will stay the same; thats a sign that the heuristic and cost
function scales match 


##### Euclidean distance
If your units can move at any angle (instead of grid directions), then you should probably use a straight line distance:
   
function heuristic(node) =  

    dx = abs(node.x - goal.x)    
    dy = abs(node.y - goal.y)     
    return D * sqrt(dx * dx + dy * dy)  
    
However, if this is the case, then you may have trouble with using A star directly because the cost function g will not match the heuristic function h. Since Euclidean distance is shorter than Manhattan or diagonal distance, you will still get shortest paths, but A star  will take longer to run. Hdnce, based on output I have changed the cost fucntion used and Heuristics to get desire results.



In [42]:
# action cost from point 1 to point 2 is terrain cost on point 2. Using this will always guarentee that our hueristic will be admissable
#i.e the cost form one to point to another would never be greater than maximum distance i.e 50
def action_cost_true((x1,y1), action, (x2,y2)):
    global terrain_cost, terrain_cost_factor
    return 1.0 + terrain_cost[(x2, y2)]/terrain_cost_factor


#manhattan distance with D=1
def heuristic_best_first((x,y), time, parents):
    dx=abs(x-goal_x)
    dy=abs(y-goal_y)
    return 1*(dx+dy)


def action_cost_best_first((x1,y1), action, (x2,y2)):
    return 1   #lazy search

beam_size_best_first = None


def heuristic_astar((x,y), time, parents):
    #same hueristic as i n bfs
    dx=abs(x-goal_x)
    dy=abs(y-goal_y)    
    return  1*(dx+dy)

def action_cost_astar((x1,y1), action, (x2,y2)):
    #returning actual cost from point 1 to point 2 is terrain cost on point 2
    return action_cost_true((x1,y1), action, (x2,y2))

#a star doesn't need beam size hence it is None
beam_size_astar = None

def heuristic_beam((x,y), time, parents):
    #Euclidean Distance
    D=1.41 # because squrt is very time consuming operation and a general value after solving euclidean distance it comes around 1.41
    dx=abs(x-goal_x)
    dy=abs(y-goal_y)
    dmax=max(dx,dy)
    dmin=min(dx,dy)
    return D* ( dmax +dmin)

def action_cost_beam((x1,y1), action, (x2,y2)):
    return action_cost_true((x1,y1), action, (x2,y2))


#beam size = number of nodes that are stored at each level of the Breadth-First Search.
beam_size_beam = 15



## Input Terrain
Following are .bmp files are input to search function. As said above, the image has scale of 50 by 50 pixels.Green pixel corresponds to intial position and blue pixel is goal point. In road-confif-tree image the red pixels represents tree or a bush which is an obstacle on the path.Also, the aldrich park has multiple intial and goal points.


                road-config-garden                     road-config-tree                    road-config-aldrich-park
<img src= "http://www.cs.colostate.edu/~pswarade/pratik_imp/s5.png"style="max-width:100%; width: 90%"> 

<img src= "http://www.cs.colostate.edu/~pswarade/pratik_imp/s3.png"style="max-width:100%; width: 27%">
                                         
                                                         road-config-side walk



In [44]:
#get all inputs and strategies to be used in respective dictionary
road_configs = ["road-config-tree-sidewalks", "road-config-garden", "road-config-aldrich-park", "road-config-tree"]
strategies = [ "best-first", "astar", "beam"]
#road_configs = ["road-config-tree-sidewalks", "road-config-garden", "road-config-aldrich-park", "road-config-tree"]
#strategies = ["best-first", "astar", "beam", "human"]

#number of people walking on terrain
iterations = 500     

# generate random seeds, so we can be sure terrain and initial/goal positions
# are reused when strategies are switched
terrain_seeds = []

for i in range(0, len(strategies)):
    terrain_seeds.append(random.random())
state_seeds = []
for i in range(0, iterations):
    state_seeds.append(random.random())

path_lengths_csv = open("path_lengths.csv", "w")
path_lengths_csv.write("road_config,strategy,iteration,path_length\n")

for road_config in road_configs:

    for i in range(0, len(strategies)):
        strategy = strategies[i]

        random.seed(terrain_seeds[i])
        #establish terrain and calculate the cost of each pixel
        establish_terrain("%s.bmp" % road_config)

        for j in range(0, iterations):
            random.seed(state_seeds[j])
            make_initial_state()
            make_goal_state() # must come second

            if strategy == "best-first":              
                (path, _, _) = search((init_x, init_y), None,
                                      heuristic_best_first, action_cost_best_first, beam_size_best_first,
                                      is_goal_state, possible_transitions, make_graphviz_node)
            elif strategy == "astar":
                (path, _, _) = search((init_x, init_y), None,
                                      heuristic_astar, action_cost_astar, beam_size_astar,
                                      is_goal_state, possible_transitions, make_graphviz_node)
            elif strategy == "beam":
                (path, _, _) = search((init_x, init_y), None,
                                      heuristic_beam, action_cost_beam, beam_size_beam,
                                      is_goal_state, possible_transitions, make_graphviz_node)

            if path is None:
                print goal_x, goal_y, init_x, init_y
            #calculate path len which is number of element in path dictionary
            print road_config, strategy, (j+1), "path length", len(path)
            path_lengths_csv.write("%s,%s,%d,%d\n" % (road_config, strategy, (j+1), len(path)))
            update_terrain_costs(path)
        # create .png image
        draw_terrain("%s-%s.png" % (road_config, strategy))

path_lengths_csv.close()
            
        


road-config-tree-sidewalks best-first 1 path length 50
road-config-tree-sidewalks best-first 2 path length 50
road-config-tree-sidewalks best-first 3 path length 46
road-config-tree-sidewalks best-first 4 path length 50
road-config-tree-sidewalks best-first 5 path length 50
road-config-tree-sidewalks best-first 6 path length 50
road-config-tree-sidewalks best-first 7 path length 50
road-config-tree-sidewalks best-first 8 path length 50
road-config-tree-sidewalks best-first 9 path length 46
road-config-tree-sidewalks best-first 10 path length 50
road-config-tree-sidewalks best-first 11 path length 46
road-config-tree-sidewalks best-first 12 path length 50
road-config-tree-sidewalks best-first 13 path length 50
road-config-tree-sidewalks best-first 14 path length 46
road-config-tree-sidewalks best-first 15 path length 50
road-config-tree-sidewalks best-first 16 path length 50
road-config-tree-sidewalks best-first 17 path length 46
road-config-tree-sidewalks best-first 18 path length 46
r

road-config-tree-sidewalks best-first 149 path length 46
road-config-tree-sidewalks best-first 150 path length 46
road-config-tree-sidewalks best-first 151 path length 46
road-config-tree-sidewalks best-first 152 path length 46
road-config-tree-sidewalks best-first 153 path length 50
road-config-tree-sidewalks best-first 154 path length 50
road-config-tree-sidewalks best-first 155 path length 46
road-config-tree-sidewalks best-first 156 path length 50
road-config-tree-sidewalks best-first 157 path length 50
road-config-tree-sidewalks best-first 158 path length 46
road-config-tree-sidewalks best-first 159 path length 46
road-config-tree-sidewalks best-first 160 path length 50
road-config-tree-sidewalks best-first 161 path length 46
road-config-tree-sidewalks best-first 162 path length 46
road-config-tree-sidewalks best-first 163 path length 46
road-config-tree-sidewalks best-first 164 path length 50
road-config-tree-sidewalks best-first 165 path length 46
road-config-tree-sidewalks best

road-config-tree-sidewalks best-first 296 path length 46
road-config-tree-sidewalks best-first 297 path length 50
road-config-tree-sidewalks best-first 298 path length 46
road-config-tree-sidewalks best-first 299 path length 46
road-config-tree-sidewalks best-first 300 path length 46
road-config-tree-sidewalks best-first 301 path length 50
road-config-tree-sidewalks best-first 302 path length 50
road-config-tree-sidewalks best-first 303 path length 50
road-config-tree-sidewalks best-first 304 path length 50
road-config-tree-sidewalks best-first 305 path length 50
road-config-tree-sidewalks best-first 306 path length 46
road-config-tree-sidewalks best-first 307 path length 50
road-config-tree-sidewalks best-first 308 path length 50
road-config-tree-sidewalks best-first 309 path length 50
road-config-tree-sidewalks best-first 310 path length 50
road-config-tree-sidewalks best-first 311 path length 46
road-config-tree-sidewalks best-first 312 path length 46
road-config-tree-sidewalks best

road-config-tree-sidewalks best-first 469 path length 46
road-config-tree-sidewalks best-first 470 path length 50
road-config-tree-sidewalks best-first 471 path length 46
road-config-tree-sidewalks best-first 472 path length 46
road-config-tree-sidewalks best-first 473 path length 50
road-config-tree-sidewalks best-first 474 path length 46
road-config-tree-sidewalks best-first 475 path length 50
road-config-tree-sidewalks best-first 476 path length 46
road-config-tree-sidewalks best-first 477 path length 50
road-config-tree-sidewalks best-first 478 path length 46
road-config-tree-sidewalks best-first 479 path length 50
road-config-tree-sidewalks best-first 480 path length 50
road-config-tree-sidewalks best-first 481 path length 46
road-config-tree-sidewalks best-first 482 path length 50
road-config-tree-sidewalks best-first 483 path length 50
road-config-tree-sidewalks best-first 484 path length 50
road-config-tree-sidewalks best-first 485 path length 46
road-config-tree-sidewalks best

road-config-tree-sidewalks astar 129 path length 76
road-config-tree-sidewalks astar 130 path length 51
road-config-tree-sidewalks astar 131 path length 55
road-config-tree-sidewalks astar 132 path length 50
road-config-tree-sidewalks astar 133 path length 55
road-config-tree-sidewalks astar 134 path length 50
road-config-tree-sidewalks astar 135 path length 50
road-config-tree-sidewalks astar 136 path length 68
road-config-tree-sidewalks astar 137 path length 50
road-config-tree-sidewalks astar 138 path length 50
road-config-tree-sidewalks astar 139 path length 50
road-config-tree-sidewalks astar 140 path length 52
road-config-tree-sidewalks astar 141 path length 50
road-config-tree-sidewalks astar 142 path length 56
road-config-tree-sidewalks astar 143 path length 50
road-config-tree-sidewalks astar 144 path length 72
road-config-tree-sidewalks astar 145 path length 50
road-config-tree-sidewalks astar 146 path length 50
road-config-tree-sidewalks astar 147 path length 69
road-config-

...

road-config-tree astar 116 path length 50
road-config-tree astar 117 path length 50
road-config-tree astar 118 path length 50
road-config-tree astar 119 path length 50
road-config-tree astar 120 path length 50
road-config-tree astar 121 path length 50
road-config-tree astar 122 path length 50
road-config-tree astar 123 path length 50
road-config-tree astar 124 path length 50
road-config-tree astar 125 path length 50
road-config-tree astar 126 path length 50
road-config-tree astar 127 path length 50
road-config-tree astar 128 path length 50
road-config-tree astar 129 path length 50
road-config-tree astar 130 path length 50
road-config-tree astar 131 path length 50
road-config-tree astar 132 path length 50
road-config-tree astar 133 path length 50
road-config-tree astar 134 path length 50
road-config-tree astar 135 path length 50
road-config-tree astar 136 path length 50
road-config-tree astar 137 path length 50
road-config-tree astar 138 path length 50
road-config-tree astar 139 path le

road-config-tree astar 312 path length 50
road-config-tree astar 313 path length 50
road-config-tree astar 314 path length 50
road-config-tree astar 315 path length 50
road-config-tree astar 316 path length 50
road-config-tree astar 317 path length 50
road-config-tree astar 318 path length 50
road-config-tree astar 319 path length 50
road-config-tree astar 320 path length 50
road-config-tree astar 321 path length 50
road-config-tree astar 322 path length 50
road-config-tree astar 323 path length 50
road-config-tree astar 324 path length 50
road-config-tree astar 325 path length 50
road-config-tree astar 326 path length 50
road-config-tree astar 327 path length 50
road-config-tree astar 328 path length 50
road-config-tree astar 329 path length 50
road-config-tree astar 330 path length 50
road-config-tree astar 331 path length 50
road-config-tree astar 332 path length 50
road-config-tree astar 333 path length 50
road-config-tree astar 334 path length 50
road-config-tree astar 335 path le

road-config-tree beam 10 path length 50
road-config-tree beam 11 path length 50
road-config-tree beam 12 path length 50
road-config-tree beam 13 path length 50
road-config-tree beam 14 path length 50
road-config-tree beam 15 path length 50
road-config-tree beam 16 path length 50
road-config-tree beam 17 path length 50
road-config-tree beam 18 path length 50
road-config-tree beam 19 path length 50
road-config-tree beam 20 path length 50
road-config-tree beam 21 path length 50
road-config-tree beam 22 path length 50
road-config-tree beam 23 path length 50
road-config-tree beam 24 path length 50
road-config-tree beam 25 path length 50
road-config-tree beam 26 path length 50
road-config-tree beam 27 path length 50
road-config-tree beam 28 path length 50
road-config-tree beam 29 path length 50
road-config-tree beam 30 path length 50
road-config-tree beam 31 path length 50
road-config-tree beam 32 path length 50
road-config-tree beam 33 path length 50
road-config-tree beam 34 path length 50


road-config-tree beam 214 path length 50
road-config-tree beam 215 path length 50
road-config-tree beam 216 path length 50
road-config-tree beam 217 path length 50
road-config-tree beam 218 path length 50
road-config-tree beam 219 path length 50
road-config-tree beam 220 path length 50
road-config-tree beam 221 path length 50
road-config-tree beam 222 path length 50
road-config-tree beam 223 path length 50
road-config-tree beam 224 path length 50
road-config-tree beam 225 path length 50
road-config-tree beam 226 path length 50
road-config-tree beam 227 path length 50
road-config-tree beam 228 path length 50
road-config-tree beam 229 path length 50
road-config-tree beam 230 path length 50
road-config-tree beam 231 path length 50
road-config-tree beam 232 path length 50
road-config-tree beam 233 path length 50
road-config-tree beam 234 path length 50
road-config-tree beam 235 path length 50
road-config-tree beam 236 path length 50
road-config-tree beam 237 path length 50
road-config-tree

road-config-tree beam 417 path length 50
road-config-tree beam 418 path length 50
road-config-tree beam 419 path length 50
road-config-tree beam 420 path length 50
road-config-tree beam 421 path length 50
road-config-tree beam 422 path length 50
road-config-tree beam 423 path length 50
road-config-tree beam 424 path length 50
road-config-tree beam 425 path length 50
road-config-tree beam 426 path length 50
road-config-tree beam 427 path length 50
road-config-tree beam 428 path length 50
road-config-tree beam 429 path length 50
road-config-tree beam 430 path length 50
road-config-tree beam 431 path length 50
road-config-tree beam 432 path length 50
road-config-tree beam 433 path length 50
road-config-tree beam 434 path length 50
road-config-tree beam 435 path length 50
road-config-tree beam 436 path length 50
road-config-tree beam 437 path length 50
road-config-tree beam 438 path length 50
road-config-tree beam 439 path length 50
road-config-tree beam 440 path length 50
road-config-tree

The above ouput shows the lenght of path for each iteration. This is also generate .csv file and write path lenght in that. 

## Results & Discussion 

   
  
  
The starting map and each final map after 500 simulated people are shown below. 
aroad-config-garden, we can see that best first search did not give any specific pass as it becomes blind search(uninformed), it just keeps expanding all the leaves and checks if one of them is the goal.Hence we get lot of path. A star on the other get all the path across the pavment or circumefernce of circle. This is because it finds shortest distance each time.Well, diameter is always smaller than circumfernce but here we are taking action cost as a variable. Hence shortest distance is along the pavement.  

In second figure, a star search and beam search does well. But beam search gives more than one path this is becuase we limit the most efficient node in expanded state to 15.Also, the tree represented as red pixel is ignored.But beam and a star search gives us accurate path where it needs to diverge than best first search. 

Third input has mutliple and multiple goal. In this case, we are randomly selecting intial state and goal states. The adjacent goal to intial state is avoided. Because of multiple goal and random of selection,we do not get exact path as for 500 iteration there are people having different goal and intial point. But you can see A star search does performs pretty good than BFS and beam search.   


In forth figure, there is two intial points and one final point with lot of obstacle. As seen in output, bfs and beam search works well. Also, A star did good job in finiding path but for each iteration it got lowest cost which indeed, I believe happened. And for second intinal that is at top, the path overalap black pavment in figure
<img src= "http://www.cs.colostate.edu/~pswarade/pratik_imp/o1.png"style="max-width:100%; width: 90%">
<img src= "http://www.cs.colostate.edu/~pswarade/pratik_imp/o2.png"style="max-width:100%; width: 90%">
<img src= "http://www.cs.colostate.edu/~pswarade/pratik_imp/o3.png"style="max-width:100%; width: 90%">
<img src= "http://www.cs.colostate.edu/~pswarade/pratik_imp/o4.png"style="max-width:100%; width: 90%">

## Future Scope
Well I did everything in 2D array, which represent 2D terrain image. But these problem have 3 dimensional scope ( shown below 
).As, human tendacy is to neglect path where he needs to climb, even a little bit. Hence, if we generate terrain with some elevation on terrain or on expected path then as per human behaviour program should generate alternate path. This case is also true for slope. If we have slope then human tends to go over that.
<img src= "http://www.cs.colostate.edu/~pswarade/pratik_imp/3dscene.png"style="max-width:100%; width: 40%">

## References

1) http://theory.stanford.edu/~amitp/GameProgramming/Heuristics.html#heuristics-for-grid-maps

2) http://www.softpanorama.org/People/Wall/larry_wall_articles_and_interviews.shtml
